In [86]:
def fibo(n: Int): Int = {
    n match {
        case 0 => 0
        case 1 => 1
        case _ => fibo(n-1) + fibo(n-2)
    }
}

fibo(50)

: 

In [85]:
def fibo(n: Int): Int = {
    def fiboCall(i: Int, fibo0: Int, fibo1: Int): Int = {
        if (i == n) fibo0 + fibo1
        else fiboCall(i+1, fibo0+fibo1, fibo0)
    }
    fiboCall(1, 0, 1)
}

fibo(50)

defined function fibo
res84_1: Int = -298632863

In [91]:
def fibo(n: Int): Int = {
    def fiboCall(i: Int, fibo0: Int, fibo1: Int): Int = {
        i match {
            case 0 => fibo0
            case _ => fiboCall(i-1, fibo0+fibo1, fibo0)
        }
    }
    fiboCall(n, 0, 1)
}

fibo(50)

defined function fibo
res90_1: Int = -298632863

In [99]:
def fibo(n: Int): Int = {
    def fiboCont(i: Int, cont: Int=>Int): Int=>Int = {
        i match {
            case 0 => cont
            case _ => fiboCont(i-1, (x: Int) => fiboCont(x-2, cont)(x-1))
        }
    }
    fiboCont(n, x => x)(n)
}
fibo(10)

: 

In [51]:
import scala.util.control.TailCalls._

/*
   Exercise 3: Ping-pong Ackermann function

   Given a function p and positive integer a and b, implement this function `ppa`: (0 <= n <= 10^6)
   ppa(p, a, b) = p(a, b)               (if a <= 0 or b <= 0)
   ppa(p, a, b) = p(a, ppa(p, a-1, b))  (if p(a, b) is even and a, b > 0)
   ppa(p, a, b) = p(ppa(p, a, b-1), b)  (if p(a, b) is odd and a, b > 0)


   Hint 1: If n > 0, wrap remained calculations, make it as an anonymous function and pass it to the parameter recursively.
           That anonymous function is called 'Continuation'. You'd better search 'Continuation Passing Style'.

   Hint 2: Due to the limits of Scala's platform (JVM), tail call (not tail recursion) is not optimized generally,
           especially when using continuation passing style.
           To optimize tail call properly, use `tailcall`, `done`, `result` functions from scala.util.control.TailCalls._
           (See https://stackoverflow.com/questions/16539488/why-scala-doesnt-make-tail-call-optimization)
   */

def ppa(p: (Int, Int)=>Int, a: Int, b: Int): Int = {
    def ppaCont(a: Int, b: Int, cont: (Int, Int) => TailRec[Int]): TailRec[Int] = {
        if (a <= 0 || b <= 0) cont(a, b)
        else if (p(a, b) % 2 == 0) ppaCont(a-1, b, (x: Int, y: Int) => tailcall(cont(a, p(x, y))))
        else ppaCont(a, b-1, (x: Int, y: Int) => tailcall(cont(p(x, y), b)))
    }
    ppaCont(a, b, (x: Int, y: Int) => done(p(x, y))).result
}
ppa((a, b) => a + b, 1000000, 1000000) //== 20100
ppa((a, b) => 3*a*a - b, 20, 10) //== 347099274

import scala.util.control.TailCalls._

/*
   Exercise 3: Ping-pong Ackermann function

   Given a function p and positive integer a and b, implement this function `ppa`: (0 <= n <= 10^6)
   ppa(p, a, b) = p(a, b)               (if a <= 0 or b <= 0)
   ppa(p, a, b) = p(a, ppa(p, a-1, b))  (if p(a, b) is even and a, b > 0)
   ppa(p, a, b) = p(ppa(p, a, b-1), b)  (if p(a, b) is odd and a, b > 0)


   Hint 1: If n > 0, wrap remained calculations, make it as an anonymous function and pass it to the parameter recursively.
           That anonymous function is called 'Continuation'. You'd better search 'Continuation Passing Style'.

   Hint 2: Due to the limits of Scala's platform (JVM), tail call (not tail recursion) is not optimized generally,
           especially when using continuation passing style.
           To optimize tail call properly, use `tailcall`, `done`, `result` functions from scala.util.control.TailCalls._
           (See https://stackoverflow.com/questions/16539488/why-sca

In [39]:
sealed abstract class BTree
case class Leaf() extends BTree
case class Node(value: Int, left: BTree, right: BTree) extends BTree

val t = Node(3, Node(1, Leaf(), Leaf()), Node(2, Leaf(), Node(4, Leaf(), Leaf())))

defined class BTree
defined class Leaf
defined class Node
t: Node = Node(
  3,
  Node(1, Leaf(), Leaf()),
  Node(2, Leaf(), Node(4, Leaf(), Leaf()))
)

### TODO: `find` as tail recursion

In [41]:
def find(t: BTree, i: Int) : Boolean =
    t match {
        case Leaf() => false
        case Node(n, _, _) if n == i => true
        case Node(_, lt, rt) => find(lt, i) || find(rt, i)
    }
find(t, 1)

defined function find
res40_1: Boolean = true

* lt, rt를 넘기면 임의의 개수의 남아 있는 트리가 생긴다. => 리스트를 사용
* continuation을 사용할 수 있지만 비효율적 (코드, closure 메모리)

In [42]:
def t: BTree = Node(5,Node(4,Node(2,Leaf(),Leaf()),Leaf()),
Node(7,Node(6,Leaf(),Leaf()),Leaf()))
find(t,7)
find(t,1)

defined function t
res41_1: Boolean = true
res41_2: Boolean = false

In [44]:
sealed abstract class BTList
case class BTNil() extends BTList
case class BTCons(hd: BTree, tl: BTList) extends BTList
def find(t: BTree, x: Int) : Boolean = {
    def findIter(ts: BTList) : Boolean =
        ts match {
            case BTNil() => false
            case BTCons(Leaf(), tl) => findIter(tl)
            case BTCons(Node(v, _, _), _) if v == x => true
            case BTCons(Node(_, l, r), tl) =>
                findIter(BTCons(l, BTCons(r, tl)))
        }
        findIter(BTCons(t, BTNil()))
}

def genTree(v: Int, n: Int) : BTree = {
    def genTreeIter(t: BTree, m : Int) : BTree =
        if (m == 0) t
        else genTreeIter(Node(v, t, Leaf()), m-1)
    genTreeIter(Leaf(), n)
}

genTree(999, 10)
find(genTree(0, 100000), 1)

defined class BTList
defined class BTNil
defined class BTCons
defined function find
defined function genTree
res43_5: BTree = Node(
  999,
  Node(
    999,
    Node(
      999,
      Node(
        999,
        Node(
          999,
          Node(
            999,
            Node(
              999,
              Node(999, Node(999, Node(999, Leaf(), Leaf()), Leaf()), Leaf()),
              Leaf()
            ),
            Leaf()
          ),
          Leaf()
        ),
        Leaf()
      ),
      Leaf()
    ),
    Leaf()
  ),
  Leaf()
)
res43_6: Boolean = false

### Typed Programming
* ***type checking at compiler time***
  * static type checking
    * Higher readability
    * ex) Rust (internally unsafe for library developers, externally strict for lang users)
    
### Untyped Programming
* ***no checking at compile time***, high efficiency
  * dynamic type checking
    * check type at run time
    * ex) Python
  * no type checking
    * propagate errors
    * hard to debug
    * not a good programming language (like javascript, not javascript)
    
### ***Why needs compile-time type checking?***
  * no type errors at run time
  * no garbage collector, no free function (소유권 규칙)

---
### Parametric Polymorphism
* same as generic in java

In [45]:
def id[A](x: A): A = x

id[String]("abc")
id(3)

defined function id
res44_1: String = "abc"
res44_2: Int = 3

In [47]:
val x = id[String] _ 
x("abc")

x: String => String = ammonite.$sess.cmd46$Helper$$Lambda$2658/1295267514@6b5609ae
res46_1: String = "abc"

In [47]:
// only monomorphic function
val y = id _ 
y("abc")

cmd47.sc:2: type mismatch;
 found   : String("abc")
 required: Nothing
val res47_1 = y("abc")
                ^Compilation Failed

: 

In [48]:
def applyn[A](f: A => A, n: Int, x: A): A =
    n match {
        case 0 => x
        case _ => f(applyn(f, n - 1, x))
    }

defined function applyn

In [49]:
// Tail Call Optimization
def applyn[A](f: A => A, n: Int, x: A): A = {
    def applynIter(i: Int, acc: A): A = {
        if (i == 0) acc
        else applynIter(i-1, f(acc))
    }
    applynIter(n, x)
}    

applyn((x:Int)=>x+1, 100, 3)
applyn((x:String)=>x+"!", 10, "gil")
applyn(id[String], 10, "hur")

defined function applyn
res48_1: Int = 103
res48_2: String = "gil!!!!!!!!!!"
res48_3: String = "hur"

In [52]:
def foo[A,B](f: A=>A, x: (A, B)) : (A, B) =
    (applyn[A](f, 10, x._1), x._2)

foo[String,Int]((x:String)=>x+"!",("abc",10))

defined function foo
res51_1: (String, Int) = ("abc!!!!!!!!!!", 10)

#### How to make `applyn` as value?
* `apply`: a special alias
* apply keyword can be removed (like `def __call__` in python)

In [66]:
// val f : [A](A=>A, Int, A) => A = applyn _
type Applyn = {def apply[A](f: A=>A, n: Int, x: A): A}

val appfn: Applyn = new {
    def apply[A](f: A=>A, n: Int, x: A): A = applyn(f, n, x)
}

def foo(f: Applyn) = {
    f.apply[String]((x:String)=>x+"!", 10, "gil")
}

def gee(f: Applyn) = {
    f((x:Int)=>x+1, 100, 3)
   // f((x:Int)=>x+1, 100, 3)
}

foo(appfn)
gee(appfn)

appfn((x:Int)=>x, 1, -1)

defined type Applyn
appfn: Applyn = ammonite.$sess.cmd65$Helper$$anon$1@572691d9
defined function foo
defined function gee
res65_4: String = "gil!!!!!!!!!!"
res65_5: Int = 103
res65_6: Int = -1

In [68]:
val fff = applyn[Int] _
appfn((x:Int)=>x, 1, -1)

fff: (Int => Int, Int, Int) => Int = ammonite.$sess.cmd67$Helper$$Lambda$3012/1045734913@795fbb6b
res67_1: Int = -1

In [71]:
def foo(f: Applyn): String = {
    val a: String = f[String]((x:String)=>x+"!", 10, "gil")
    val b: Int = f[Int]((x:Int)=>x+2, 10, 5)
    a + b.toString()
}
foo(appfn)

defined function foo
res70_1: String = "gil!!!!!!!!!!25"